In [1]:
from torchvision import datasets, transforms
import torch
from net import AlexNetPlusLatent
import numpy as np

In [2]:
bits = 48
pretrained = 91.33

In [3]:
net = AlexNetPlusLatent(bits)
net.load_state_dict(torch.load('./model/91.33'))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Use device: ' + str(device))
net.to(device)
net.eval()

Use device: cuda:0


AlexNetPlusLatent(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (remain): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p

In [4]:
def load_data():
    transform_test = transforms.Compose(
        [
         transforms.Resize(227),
         transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ]
    )
    testset = datasets.CIFAR10(root='./data', train=False, download=True,
                               transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset)
    return testloader

In [5]:
def int32_aggregate_output(_loader):    
    #var intitialization
    _output = []
    
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(_loader):
            _int = int32_single_output(inputs)
            b = (batch_idx, _int, int(targets))
            if batch_idx%500==0:
                print('.', end='')
            _output.append(b)
        return _output

In [6]:
def find_picutre(given_pic_id, _list, _id_place=0):
    _output = [item for item in _list if item[_id_place] == given_pic_id]
    return _output[0]

In [7]:
def single_output(data):
    inputs = data.to(device)
    outputs, _ = net(inputs)
    return outputs.data

In [8]:
def int32_single_output(_data):
    _tensor = (single_output(_data)[0,:]>0.5).int()
    _str = ''
    for i in range(0, _tensor.shape[0]):
        _str += str(int(_tensor[i]))
    _int = int(_str, 2)
    return _int

In [9]:
_loader = load_data()
a = int32_aggregate_output(_loader)

Files already downloaded and verified
....................

In [10]:
a[6569][1]

31598448389273

In [11]:
find_picutre(6569, a)

(6569, 31598448389273, 8)

In [12]:
def hammingDistance(x, y):
    ans = 0
    for i in range(31,-1,-1):
        b1= x>>i&1
        b2 = y>>i&1
        ans+= not(b1==b2)
    return ans

In [13]:
def tpr(hash_list, _given_pic_class, ):
    #var initialization
    _tp = 0
    _fn = 0
    
    for (_, _, _list_pic_class) in hash_list:
        #identify TP, FP, FN
        if _given_pic_class == _list_pic_class:
            _tp += 1
        else:
            _fn += 1
    
    _tpn = _tp/(_tp + _fn)
    return _tp, _fn, _tpn

In [14]:
def overal_AP(hash_list, _given_pic_class):
    #var initialization
    _overal_AP = 0.0
    i = 0
    _tp = 0
        
    for (_, _, _list_pic_class) in hash_list:
        i += 1
        if _given_pic_class == _list_pic_class:
            _tp += 1
            _overal_AP += _tp/i
        
    return _overal_AP/_tp
        

In [15]:
def n_most_relative(_given_pic_id, hash_list, _num_of_pictures):
    #find given picture
    _given_pic = find_picutre(_given_pic_id, hash_list)
    
    #var initialization
    _output = []
    
    #extract given picture properties
    _given_pic_hash = _given_pic[1]
    _given_pic_class = _given_pic[2]
      
    for _list_pic in hash_list:        
        #extract list picture properties
        _list_pic_id = _list_pic[0]
        _list_pic_hash = _list_pic[1]
        _list_pic_class = _list_pic[2]
        
        #compute hamming distance
        _hamming_distance = hammingDistance(_given_pic_hash, _list_pic_hash)
        
        
        b = (hash_list[_list_pic_id][0],_hamming_distance, _list_pic_class)
        _output.append(b)
        
    #sort and crop output
    _output = sorted(_output, key=lambda tup: tup[1])[0: _num_of_pictures]
    
    #compute TP, FN, TPN
    _tp, _fn, _tpn = tpr(_output, _given_pic_class)
    
    #compute Overal AP
    _overal_AP = overal_AP(_output, _given_pic_class)
    
    return _overal_AP, _output

In [17]:
c = n_most_relative(2342 , a, 500)
c

(0.9862851014125946,
 [(16, 0, 5),
  (31, 0, 5),
  (39, 0, 5),
  (42, 0, 5),
  (61, 0, 3),
  (101, 0, 5),
  (141, 0, 5),
  (155, 0, 5),
  (168, 0, 5),
  (181, 0, 5),
  (190, 0, 5),
  (207, 0, 5),
  (212, 0, 5),
  (230, 0, 5),
  (238, 0, 5),
  (239, 0, 5),
  (250, 0, 5),
  (318, 0, 5),
  (319, 0, 5),
  (327, 0, 5),
  (361, 0, 5),
  (389, 0, 5),
  (431, 0, 5),
  (444, 0, 5),
  (457, 0, 5),
  (460, 0, 5),
  (484, 0, 5),
  (510, 0, 5),
  (514, 0, 5),
  (525, 0, 5),
  (533, 0, 5),
  (544, 0, 5),
  (549, 0, 5),
  (553, 0, 5),
  (579, 0, 5),
  (601, 0, 5),
  (614, 0, 5),
  (679, 0, 5),
  (699, 0, 5),
  (729, 0, 5),
  (768, 0, 5),
  (817, 0, 5),
  (899, 0, 5),
  (903, 0, 5),
  (924, 0, 5),
  (1000, 0, 5),
  (1041, 0, 5),
  (1045, 0, 5),
  (1055, 0, 5),
  (1084, 0, 5),
  (1097, 0, 5),
  (1116, 0, 5),
  (1117, 0, 5),
  (1214, 0, 5),
  (1235, 0, 5),
  (1252, 0, 5),
  (1276, 0, 5),
  (1283, 0, 5),
  (1290, 0, 5),
  (1316, 0, 5),
  (1318, 0, 5),
  (1337, 0, 5),
  (1362, 0, 5),
  (1400, 0, 5),
  (15